从activatemq接收用户更新的文章数据，放到hbase中

In [1]:
class Conf:
    pass

conf = Conf()
conf.activemq_queue = "videobase-add"
conf.hbase_table = "videobase_all"
conf.__dict__

{'activemq_queue': 'videobase-add', 'hbase_table': 'videobase_all'}

In [24]:
import json
import time
json_str = """
{"publishTime":1477856760000,"categorys":["来源-测试"],"updateTime":1477856760000,"source":"着迷网","title":"SKT登上历史之巅，熊猫直播全程记录三冠王加冕","content":"2016英雄联盟全球总决赛今日迎来了全球瞩目的最终决赛，由SKT和SSG为全球玩家上演了一场BO5的精彩绝伦的对决，SKT与SSG打满5场，在此过程中战局形势扣人心弦，而最终SKT的顽强的意志与恐怖的实力，最终夺得S6总冠军。熊猫直播全程见证三冠王的诞生，为中国玩家送上了第一手的赛场直播。 前两局SKT都在遇到不小的阻力下拿下比赛，当所有人都觉得，以SKT前两局的表现来说，能够以统治力的实力直接3：0带走SSG的时候，SSG通过稳定运营最终双方经过71分钟的鏖战，SSG成功续命，第四局SSG士气大振再下一城，双方进入最终决胜局。SSG选择拿风龙的那一刻让他们失去了持续滚雪球的资本，随后SKT再次拿下大龙上高，这一次直接三路全破，SKT最终堵泉磨掉SSG的门牙，超级兵进场的那一刻，SKT赢下了最终局。在此SKT成功拿下三冠王。熊猫直播间也在此时达到高潮，弹幕刷爆整个直播间。 熊猫直播在今天的直播中，观众数已达到了最高，从700万在线一路爬升到了近1100万，熊猫直播间的火热不仅是玩家们对SKT能否夺得三冠王的期待，更是熊猫直播平台本身贴心专业的体现。全程实时高清的直播流，让弹幕大神能够全体狂欢。 1000万人成功见证了SKT创造的全新历史，熊猫直播间以及现场无数的粉丝为其表现和他取得的成绩而呐喊，现场解说席内的熊猫直播签约主播 PDD也真诚祝福了SKT，并再次为SSG报以掌声，可以说S6总决赛是一场令所有人都为之震撼的世纪大战。 2016年英雄联盟世界总决赛终于划上句点，熊猫直播在此过程中的全程直播获得了几千万的观看人次，每一场精彩比赛，都在第一时间让玩家同步赛况，直达比赛现场，这也是直播平台为大家提供的最佳最为便利的服务，熊猫直播将一如既往的为大家带来最为精彩的直播内容，全明星赛将在12月8日-12月11日在巴塞罗那举办，届时大家记得如期关注熊猫直播，观战你最喜欢的明星选手玩转全明星。 虽然本届总决赛LPL地区战队未能晋级决赛，但我们喜欢的选手及战队，仍然有机会为大家带来精彩的比赛内容，让我们为自己的喜欢选手投上宝贵的一票，希望大家能够多多支持熊猫直播签约战队RNG，我们全明星赛不见不散。最后，再次恭喜SKT，登上历史之巅。","tags":["题材","娱乐","冒险","社会"],"createTime":1477856760000,"qualityScore":0.0,"publisher":{"id":"0000000001"},"clickNum":0,"id":"471799452","state":0,"articleAuthor":"zhangsan"}
"""
jobj = json.loads(json_str)
time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime(jobj['createTime']/1000))

'2016-10-31-03-46-00'

In [25]:
import time
import happybase
import json

conn = happybase.Connection('localhost', autoconnect=True)
conn.open()
table = conn.table(conf.hbase_table)
def parse_save_hbase(line):
    jobj = json.loads(line)
    table.put(row=jobj['id'],
              data={
                  "d:id": jobj['id'],
                  "d:title":jobj['title'],
                  "d:content":jobj['content'],
                  "d:state":str(jobj['state']),
                  "d:createTime":time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime(jobj['createTime']/1000)),
                  "d:updateTime":time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime(jobj['updateTime']/1000)),
                  "d:publishTime":time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime(jobj['publishTime']/1000)),
                  "d:source":jobj['source'],
                  "d:articleAuthor":jobj['articleAuthor'],
                  "d:categorys":",".join(jobj['categorys']),
                  "d:clickNum":str(jobj['clickNum']),
                  "d:qualityScore":str(jobj['qualityScore'])
              },
              timestamp=2,
              wal=False)
table.families()

{b'd': {'name': b'd:',
  'max_versions': 1,
  'compression': b'NONE',
  'in_memory': False,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'bloom_filter_nb_hashes': 0,
  'block_cache_enabled': False,
  'time_to_live': 2147483647}}

In [26]:
import time
import stomp
class TestListener(stomp.ConnectionListener):
    def on_message(self, msg):
        print(msg.body)
        parse_save_hbase(msg.body)

conn = stomp.Connection10()
conn.set_listener(conf.activemq_queue, TestListener())
conn.connect()
conn.subscribe(conf.activemq_queue)
print("开始接收",time.time())
time.sleep(10)

开始接收 1639150212.603908
{"publishTime":1477808700000,"categorys":["来源-测试"],"updateTime":1477808700000,"source":"微文化联盟官微","title":"为你朗诵||小护士","content":" 问题：怎么才能每天都收到这种文章呢？答案：只需要点击图片上边的《微文化联盟官微》关注即可 微文化联盟官微 微文化联盟招聘朗诵主播10名，欢迎有更多朗诵特长的嘉宾加盟微文化联盟，共同创建文学艺术心灵家园！ 报名者请加微信号：zxasqw4399 点开下面绿色开关聆听主播郭慧精彩朗诵《小护士》作者：林昔 朗诵者：郭慧 洁白的燕尾帽呢喃着医院的晨露轻声细语的问候柔柔的飘荡在病区小护士你的身影温和的微笑点燃了沉寂的失望让岁月有了热爱的渴求 小护士沉沉浮浮的人生已走在了辉煌的顶峰至今在您的面前却是一张没有污点的白纸你的沉默最能慰藉追逐一生的归零小护士原谅此刻的我脆弱的像个婴儿将你视为黑暗的明亮火柴 小护士你大海般的胸怀将一滴墨包容将酸甜苦辣柔成美丽的微笑永远是一颗沧桑的心灵孵化剂小护士啊感恩生命的跌涯遇见最美的你 作者简介 林昔：原名贺雪利，陕西省宝鸡市人，医务工作者，诗歌散文散见宝鸡日报，文学爱好者，平时喜欢阅读书籍，朗诵，及写作。平台主播 郭慧，山西省长治市朗诵家协会理事，热爱朗诵艺术，希望用声音搭建与听众心灵之间的桥梁，用心感悟文字，用声音诠释美好的生活，从而与听众达成心灵上的共鸣。微信：guohui955562 荔枝Fm：1241409 【十月份稿酬领取规则】真正的好作品应流芳民间，让大众阅读，让百姓分享。凡是阅读量达500以上一稿费10元，阅读量达1000以上一稿费20元，阅读量达2000以上一稿费30元，稿费为投稿人所有，十一月份5-10号截图给云飘飘微信号zxasqw4399领取稿费，过期算自动放弃，稿费多少将于大众阅读决定，并选出精品入书刊投稿须知投稿邮箱:1821607172@qq.com①来稿请务必注明作者微信号，并关注微文化联盟，已经在微信公众号发表并且被原创保护的作品，请勿投稿本平台。②作品体裁不限，特别欢迎配乐朗诵音频、视频、音乐、现代诗歌、散文随笔，唯美爱情，智慧感悟等作品。只接受邮箱投稿。（凡是提供mp3配音的优先发表）③另欢迎励志正能量，人生感悟

In [27]:
conn.disconnect()
print("接收已停止",time.time())

接收已停止 1639150222.6208775
